# Защита персональных данных клиентов
---

#### Описание проекта

Требуется защитить данные клиентов страховой компании «Хоть потоп». Нужно разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Требуется обосновать корректность работы алгоритма. Важно защитить данные таким образом, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

#### Описание данных:

- Пол: Бинарный признак, представляющий пол клиента;
- Возраст: Возраст клиента;
- Зарплата: Годовая зарплата клиента;
- Члены семьи: Количество членов семьи;
- Страховые выплаты: Количество страховых выплат клиенту за последние 5 лет.

#### Задачи исследования:

##### Этап 1. Разведочный анализ данных (EDA):

 - Загрузка данных из файлов;

 - Первичный осмотр данных.

##### Этап 2. Обучение модели на исходных данных;

##### Этап 3. Обучение модели на преобразованных данных:

 - Теоретическое обоснование преобразования (использование данного подхода);

 - Докозательство;

 - Генерация обратимой матрицы;

 - Преобразование данных;

 - Реализация и Проверка Алгоритма;

 - Вывод.





In [ ]:
# Стандартные библиотеки
import warnings
import random

# Сторонние библиотеки и фреймворки
import pandas as pd
import numpy  as np
from   numpy.linalg import inv

# Инструменты для машинного обучения
from sklearn.model_selection import train_test_split
from sklearn.linear_model    import LinearRegression
from sklearn.metrics         import r2_score

# Настройки
pd.options.display.float_format = '{:.2f}'.format
warnings.filterwarnings("ignore")
RANDOM_STATE = random.seed(42)

/opt/conda/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
def generate_invertible_matrix(size:int):
    """
    Функция для создания случайной обратимой матрицы заданного размера.

    Эта функция генерирует матрицу размера 'size x size' с элементами, случайно выбранными
    из равномерного распределения от 0 до 1. После генерации проверяется, обратима ли матрица.
    Если матрица обратима, она возвращается как результат функции. В противном случае
    процесс генерации повторяется до получения обратимой матрицы.

    Параметры:
    size : int
        Размерность генерируемой матрицы (количество строк и столбцов).

    Возвращает:
    matrix : ndarray
        Случайно сгенерированная обратимая матрица размера 'size x size'.

    Исключения:
    np.linalg.LinAlgError
        Исключение возникает, если генерируемая матрица необратима. В таком случае
        функция повторно пытается сгенерировать другую матрицу.
    """
    while True:
        matrix = np.random.rand(size, size)
        try:
            inv(matrix)
            break
        except np.linalg.LinAlgError:
            pass
    return matrix

#### Этап 1. Разведочный анализ данных (EDA)

In [ ]:
try:
    df = pd.read_csv('/content/insurance.csv', sep = ',')
except:
    df = pd.read_csv('/datasets/insurance.csv', sep = ',')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [ ]:
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.00,49600.00,1,0
1,0,46.00,38000.00,1,1
2,0,29.00,21000.00,0,0
3,0,21.00,41700.00,2,0
4,1,28.00,26100.00,0,0


In [ ]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Пол,5000.00,0.50,0.50,0.00,0.00,0.00,1.00,1.00
Возраст,5000.00,30.95,8.44,18.00,24.00,30.00,37.00,65.00
Зарплата,5000.00,39916.36,9900.08,5300.00,33300.00,40200.00,46600.00,79000.00
Члены семьи,5000.00,1.19,1.09,0.00,0.00,1.00,2.00,6.00
Страховые выплаты,5000.00,0.15,0.46,0.00,0.00,0.00,0.00,5.00


#### Этап 2. Обучение модели на исходных данных

In [ ]:
# Разделение данных на признаки и целевую переменную
X = df.drop('Страховые выплаты', axis=1)
y = df['Страховые выплаты']

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Обучение модели линейной регрессии на исходных данных
model = LinearRegression()
model.fit(X_train, y_train)
predictions = model.predict(X_test)
original_r2 = r2_score(y_test, predictions)

#### Этап 3. Обучение модели на преобразованных данных

#### Теоретическое обоснование преобразования

##### Влияние на Качество Линейной Регрессии

Умножение признаков на обратимую матрицу не изменит качество линейной регрессии. Это связано с тем, что в линейной регрессии ищется решение вида
$y=Xw+ϵ$, где $X$ - матрица признаков, $w$ - вектор весов, а $ϵ$ - ошибка. Если признаки умножить на обратимую матрицу $A$, то задача преобразуется в $y=(XA)w′+ϵ$. Поскольку $A$ обратима, можно найти новый вектор весов $w′$, который компенсирует это преобразование.

##### Связь Параметров Линейной Регрессии

Вектор весов $w′$ в преобразованной задаче связан с исходным вектором весов $w$ через обратимую матрицу $A$. Это обеспечивает эквивалентность прогнозов до и после преобразования.

### Пример Преобразования Данных в Линейной Регрессии

Предположим, у нас есть матрица признаков $X$ размером 2×2 и вектор целевых значений $y$.

Пусть:
- Матрица признаков $X$:

  $
  X = \begin{pmatrix}
  1 & 2 \\
  3 & 4
  \end{pmatrix}
  $
- Вектор целевой переменной $y$:
  
  $
  y = \begin{pmatrix}
  5 \\
  11
  \end{pmatrix}
  $
- Вектор весов $w$ (коэффициенты модели):
  
  $
  w = \begin{pmatrix}
  1 \\
  1
  \end{pmatrix},
  $ так что $y = Xw$.
  
  

#### Преобразование с Обратимой Матрицей

Теперь выберем обратимую матрицу $A$ размером 2×2 (например, любую квадратную матрицу с ненулевым определителем).

Пусть:

- Обратная матрица $A$ размером 2×2:

  $
  A = \begin{pmatrix}
  2 & 0 \\
  0 & 2
  \end{pmatrix}
  $

- Тогда ($A^{-1}$):

  $
  A^{-1} = \begin{pmatrix}
  0.5 & 0 \\
  0 & 0.5
  \end{pmatrix}
  $

#### Преобразование Признаков

- Умножаем матрицу признаков $X$ на матрицу $A$:

  $
  XA = \begin{pmatrix}
  1 & 2 \\
  3 & 4
  \end{pmatrix} \times \begin{pmatrix}
  2 & 0 \\
  0 & 2
  \end{pmatrix} = \begin{pmatrix}
  2 & 4 \\
  6 & 8
  \end{pmatrix}
  $

#### Нахождение Новых Весов

- Вычисляем новые веса $w$:

  $
  w' = A^{-1}w = \begin{pmatrix}
  0.5 & 0 \\
  0 & 0.5
  \end{pmatrix} \times \begin{pmatrix}
  1 \\
  1
  \end{pmatrix} = \begin{pmatrix}
  0.5 \\
  0.5
  \end{pmatrix}
  $

#### Применение Новых Весов

- Вычисляем прогнозы с новыми признаками и весами:
  
  $
  y = XA \times w' = \begin{pmatrix}
  2 & 4 \\
  6 & 8
  \end{pmatrix} \times \begin{pmatrix}
  0.5 \\
  0.5
  \end{pmatrix} = \begin{pmatrix}
  5 \\
  11
  \end{pmatrix}
  $

### Доказательство Неизменности Предсказаний Линейной Регрессии После Преобразования

#### Исходные Уравнения

1. Определяем, что предсказания модели линейной регрессии задаются как $y = Xw$, где:
   - $X$ — матрица признаков,
   - $w$ — вектор весов модели,
   - $y$ — вектор предсказаний модели.

#### Преобразование с Обратимой Матрицей

2. Предполагаем, что после преобразования признаков уравнение изменяется на $y = XA * w'$, где:
   - $A$ — обратимая матрица,
   - $w$ — новый вектор весов после преобразования.

#### Шаги Доказательства

3. Из $y = Xw$ и $y = XA * w'$ следует, что $Xw = XA * w'$:

   $
   y = Xw \quad \text{и} \quad y = XA * w' \quad \Rightarrow \quad Xw = XA * w'
   $

4. Принимая во внимание, что $w' = A^{-1}w$, мы можем заменить $w'$ в уравнении:

   $
   Xw = XA * A^{-1}w
   $

5. Умножение матрицы $A$ на её обратную $A^{-1}$ даёт единичную матрицу $I$:

   $
   Xw = XA * A^{-1}w \quad \Rightarrow \quad Xw = Xw * AA^{-1}
   $

6. Так как $AA^{-1}$ равно единичной матрице $I$, уравнение упрощается до:
   
   $
   Xw = Xw * AA^{-1} \quad \Rightarrow \quad Xw = XwI
   $

7. Умножение на единичную матрицу $I$ не изменяет исходную матрицу:
   
   $
   Xw = XwI \quad \Rightarrow \quad Xw = Xw
   $


#### Вывод

Таким образом, умножение признаков на обратимую матрицу $A$ и последующая корректировка весов
$w′$ позволяет сохранить исходные предсказания модели, что подтверждает теорию о неизменности качества линейной регрессии при таком преобразовании.

---
### Важно помнить: Формула для Нахождения Весов в Линейной Регрессии

Формула, используемая для вычисления весов $w$ в линейной регрессии с использованием метода наименьших квадратов, выглядит следующим образом:

$ w = (X^TX)^{-1}X^Ty $

где:

- $X$ — матрица признаков;
- $X^T$ — транспонированная матрица признаков;
- $y$ — вектор целевой переменной.



#### Предложение Алгоритма Преобразования

##### Алгоритм будет включать следующие шаги:

1.Генерация случайной обратимой матрицы $A$ соответствующего размера;

2.Умножение матрицы признаков $X$ на $A$ для получения преобразованных данных.

#### Реализация и Проверка Алгоритма

Теперь мы реализуем этот алгоритм и проверим, сохраняется ли качество модели линейной регрессии (используя метрику R2) до и после преобразования. Мы начнем с генерации обратимой матрицы. ​

In [ ]:
# Генерация обратимой матрицы
matrix_size = X.shape[1]
A = generate_invertible_matrix(matrix_size)

In [ ]:
# Преобразование данных
X_train_transformed = X_train.dot(A)
X_test_transformed = X_test.dot(A)

In [ ]:
# Обучение модели на преобразованных данных
model_transformed = LinearRegression()
model_transformed.fit(X_train_transformed, y_train)
predictions_transformed = model_transformed.predict(X_test_transformed)
transformed_r2 = r2_score(y_test, predictions_transformed)

In [ ]:
print('Метрика R2 на исходных данных ', original_r2)
print('Метрика R2 на преобразованных данных ', transformed_r2)

Метрика R2 на исходных данных  0.43686949231379935
Метрика R2 на преобразованных данных  0.4368694923119707


#### Результаты Проверки Алгоритма

##### Коэффициент детерминации R2 для модели линейной регрессии:

- На исходных данных: 0.4369;
- На преобразованных данных: 0.4369

Эти результаты показывают, что качество модели линейной регрессии не изменилось после преобразования данных. Коэффициент R2 остался почти идентичным, что подтверждает теоретическое предположение о том, что умножение признаков на обратимую матрицу не влияет на способность модели линейной регрессии делать точные предсказания.

#### Выводы

Предложенный метод преобразования данных путём умножения на обратимую матрицу оказался эффективным для защиты личной информации клиентов. Он изменяет данные таким образом, что становится сложно восстановить исходную личную информацию, одновременно сохраняя важные для моделирования статистические свойства данных. Этот подход может быть особенно полезен в области страхования для защиты конфиденциальных данных клиентов, не ухудшая при этом качества аналитических моделей.